# notebooks.smart_microfluidics_models_extended

## Setup

In [1]:
!git clone https://github.com/leonardoLavagna/smart_microfluidics

Cloning into 'smart_microfluidics'...
remote: Enumerating objects: 865, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 865 (delta 71), reused 15 (delta 15), pack-reused 744 (from 3)
Receiving objects: 100% (865/865), 4.06 MiB | 9.13 MiB/s, done.
Resolving deltas: 100% (519/519), done.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import numpy as np

## Get Data

In [7]:
file_path = "smart_microfluidics/data/extended_data.csv"
data = pd.read_csv(file_path, encoding="latin-1")
yes_data = data[data["OUTPUT"] == 1]

## Random forest regressor

In [8]:
features = yes_data.drop(columns=["ID", "SIZE", "PDI", "OUTPUT"])
targets = yes_data[["SIZE", "PDI"]]
categorical_columns = features.select_dtypes(include=["object"]).columns
numerical_columns = features.select_dtypes(include=["float64", "int64"]).columns

preprocessor = ColumnTransformer(transformers=[("cat", OneHotEncoder(drop="first"), categorical_columns),],remainder="passthrough")
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
model = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))])
model.fit(X_train, y_train)
pickle_file_path = "random_forest_model_extended_1.pkl"
with open(pickle_file_path, "wb") as file:
    pickle.dump(model, file)

print(f"Model saved to {pickle_file_path}")

Model saved to random_forest_model_extended_1.pkl


In [9]:
with open("random_forest_model_extended_1.pkl", "rb") as file:
    loaded_model = pickle.load(file)
input_data = pd.DataFrame({
    "ML": ["HSPC"],
    "CHIP": ["Micromixer"],
    "TLP": [5.0],
    "ESM": [0.0],
    "HSPC": [3.75],
    "CHOL": [0.0],
    "PEG": [1.25],
    "TFR ": [1.0],
    "FRR": [3.0],
    "BUFFER": ["PBS"],
    "OUTPUT": [1]
})

predictions = loaded_model.predict(input_data)
print("Predicted SIZE and PDI:", predictions)


Predicted SIZE and PDI: [[120.03      0.35539]]


## XGBoost

In [10]:
features = yes_data.drop(columns=["ID", "SIZE", "PDI"])
targets = yes_data[["SIZE", "PDI"]]
categorical_columns = features.select_dtypes(include=["object"]).columns
numerical_columns = features.select_dtypes(include=["float64", "int64"]).columns

preprocessor = ColumnTransformer(transformers=[("cat", OneHotEncoder(drop="first"), categorical_columns),],remainder="passthrough")
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
xgboost_model = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", MultiOutputRegressor(XGBRegressor(n_estimators=100, random_state=42)))])

xgboost_model.fit(X_train, y_train)
pickle_file_path = "xgboost_model_extended_1.pkl"
with open(pickle_file_path, "wb") as file:
    pickle.dump(xgboost_model, file)
print(f"Model saved to {pickle_file_path}")

Model saved to xgboost_model_extended_1.pkl


In [11]:
with open("xgboost_model_extended_1.pkl", "rb") as file:
    loaded_model = pickle.load(file)
input_data = pd.DataFrame({
    "ML": ["HSPC"],
    "CHIP": ["Micromixer"],
    "TLP": [5.0],
    "ESM": [0.0],
    "HSPC": [3.75],
    "CHOL": [0.0],
    "PEG": [1.25],
    "TFR ": [1.0],
    "FRR": [3.0],
    "BUFFER": ["PBS"],
    "OUTPUT": [1]
})

predictions = loaded_model.predict(input_data)
print("Predicted SIZE and PDI:", predictions)


Predicted SIZE and PDI: [[118.0047       0.33534396]]


## Inverse problem

In [12]:
targets = yes_data[["SIZE", "PDI"]]
numerical_columns = ["ESM", "HSPC", "CHOL", "PEG", "TFR ", "FRR"]
features = yes_data[numerical_columns]
X_train, X_test, y_train, y_test = train_test_split(targets, features, test_size=0.2, random_state=42)

inverse_model = MultiOutputRegressor(XGBRegressor(n_estimators=100, random_state=42))
inverse_model.fit(X_train, y_train)

pickle_file_path = "inverse_xgboost_model_extended_1.pkl"
with open(pickle_file_path, "wb") as file:
    pickle.dump(inverse_model, file)

print(f"Inverse model saved to {pickle_file_path}")


Inverse model saved to inverse_xgboost_model_extended_1.pkl


In [13]:
with open("inverse_xgboost_model_extended_1.pkl", "rb") as file:
    inverse_model = pickle.load(file)
input_data = pd.DataFrame({"SIZE": [118.0], "PDI": [0.33]})

predictions = inverse_model.predict(input_data)
predictions = np.abs(predictions)
predictions = np.where(predictions < 0.5, 0, predictions)
print("Predicted values for ESM, HSPC, CHOL, PEG, TFR, and FRR:", predictions)

Predicted values for ESM, HSPC, CHOL, PEG, TFR, and FRR: [[0.        3.774202  0.        1.249703  0.9808568 3.0403361]]
